In [0]:
spark.sql("USE default")


DataFrame[]

In [0]:
#Bronze layer (Raw Data)
from pyspark.sql import functions as F

bronze_df = spark.table("customer_transactions") \
    .withColumn("ingestion_ts", F.current_timestamp())


In [0]:
#Save as Bronze Delta table
bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_customer_transactions")

In [0]:
#SILVER LAYER (CLEANED DATA)
silver_df = bronze_df \
    .filter(F.col("annual_income") > 0) \
    .dropDuplicates(["customer_id"])

In [0]:
#Save as Silver Delta table
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_customer_transactions")

In [0]:
#GOLD LAYER (BUSINESS AGGREGATES)
gold_df = silver_df.groupBy("country") \
    .agg(
        F.count("customer_id").alias("total_customers"),
        F.avg("annual_income").alias("avg_income")
    )

In [0]:
#Save as Gold Delta table:
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_customer_summary")